# Data Acquisition and Processing for Census Tracts

In [ ]:
import pandas as pd
from parameters import *
from utils import *
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
census_api_key = "85e4d04ebdc75be192b6cd5444f6dc9970a95666"

years = range(2012, 2024)
with ThreadPoolExecutor(max_workers=10) as parallel_executor:
    futures = []
    for year in years:
        for state in states:
            futures.append([year, parallel_executor.submit(get_state_tracts, state, year, census_api_key)])

In [ ]:
census_bureau_data = {}
for year in years:
    yearly_data = filter(lambda x: x[0] == year, futures)
    dfs = []
    for data in yearly_data:
        df = data[1].result()
        df['GEOID'] = df['state'].astype(str) + df['county'].astype(str) + df['tract'].astype(str)
        df = df[1:]
        # print(df.head())
        dfs.append(df)
    census_bureau_data[year] = pd.concat(dfs, axis=0, ignore_index=False)

In [ ]:
census_bureau_data[2013].head()